In [1]:
import os
import openpyxl
from shapely.geometry import Polygon
import random
import time
os.environ['USE_PYGEOS'] = '0'

Setup

In [2]:
class MBR:
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
    def __repr__(self):
        return "("+str( self.x_min)+", " + str(self.y_min) + ", " + str(self.x_max) + ", " + str(self.y_max)+")"

In [3]:
def load_data(path):
    polygons=[]
    with open(path) as dataset_file:
        rows = dataset_file.readlines()
        for row in rows[1:]:
            start_POS = row.find("POLYGON (") + 10
            end_POS = row.find("))")
            result_str = row[start_POS: end_POS]
            poly = result_str.split(',')
            polygons.append(poly)
    return polygons

def cal_mbr(poly):
    local_min_x=float("inf")
    local_min_y=float("inf")
    local_max_x=float("-inf")
    local_max_y=float("-inf")
    for polygon_point in poly:
            p_x, p_y = polygon_point.strip().split(" ")
            p_x = float(p_x)
            p_y = float(p_y)
            local_min_x = min(local_min_x, p_x)
            local_min_y = min(local_min_y, p_y)
            local_max_x = max(local_max_x, p_x)
            local_max_y = max(local_max_y, p_y)
    return MBR(local_min_x, local_min_y, local_max_x,local_max_y)

def cal_mbr_for_all(polygon_list_t):
    MBR_List_t=[]
    point_min_x=float("inf")
    point_min_y=float("inf")
    point_max_x=float("-inf")
    point_max_y=float("-inf")
    for poly in polygon_list_t:
        temp = cal_mbr(poly)
        MBR_List_t.append(temp)
        point_min_x = min(point_min_x, temp.x_min)
        point_min_y = min(point_min_y, temp.y_min)
        point_max_x = max(point_max_x, temp.x_max)
        point_max_y = max(point_max_y, temp.y_max)
    MBR_for_all_t=MBR(point_min_x, point_min_y, point_max_x,point_max_y)
    return MBR_for_all_t, MBR_List_t

def is_contain(mbr1, mbr2):
    return True  if mbr1.x_min >= mbr2.x_min and mbr1.x_max <= mbr2.x_max and mbr1.y_min >= mbr2.y_min and mbr1.y_max <= mbr2.y_max else False


Load data and calculate mbr (in ass1)

In [4]:
src_path = './data/Buildings.csv'
data = load_data(src_path)
MBR_for_all, MBR_result=cal_mbr_for_all(data)
print("MBR of all Polygons is " +str(MBR_for_all))

MBR of all Polygons is (113.9310037, 22.1973011, 114.3763554, 22.5069962)


In [5]:
polygon_list = []
for mbr in MBR_result:
    polygon_mbr = Polygon([(mbr.x_min, mbr.y_min), (mbr.x_min, mbr.y_max), (mbr.x_max, mbr.y_min), (mbr.x_max, mbr.y_max)])
    polygon_list.append(polygon_mbr)

Setup R Tree

In [6]:
class RTreeNode:
    def __init__(self, is_leaf=False, parent=None):
        self.polygons = []
        self.children = []
        self.is_leaf = is_leaf
        self.bounds = None
        self.parent_node = parent

    def insert(self, polygons, index, d, n):
        if not self.is_leaf:
            self.choose_leaf(polygons, index, d, n)
        else:
            if len(self.polygons) >= n:
                self.overflow(d, n)
                self.parent_node.insert(polygons, index, d, n)
            else:
                self.polygons.append((polygons, index))
                self.bounds = self.cal_area()

    def update_mbr(self):
        if self.parent_node is not None:
            self.parent_node.bounds = self.parent_node.cal_area()
            self.parent_node.update_mbr()

    def intersection(self, query):
        return not (query.bounds[0] > self.bounds[2] or query.bounds[2]< self.bounds[0] or query.bounds[1] > self.bounds[3] or query.bounds[3] < self.bounds[1])

    def overflow(self, d, n):
        node_temp = self.splitNode(d, n)
        self.bounds = self.cal_area()
        node_temp.bounds = node_temp.cal_area()
        if self.parent_node is not None:
            if len(self.parent_node.children) >= d:
                new_parent_node = self.parent_node.overflow(d, n)
                node_temp.parent_node = self.parent_node
                new_parent_node.children.append(node_temp)
                new_parent_node.bounds = new_parent_node.cal_area()
                return node_temp
            else:
                node_temp.parent_node = self.parent_node
                self.parent_node.children.append(node_temp)
                return node_temp
        else:
            parent_new_node = RTreeNode(is_leaf=False)
            node_temp.parent_node = parent_new_node
            self.parent_node = parent_new_node
            parent_new_node.children.append(self)
            parent_new_node.children.append(node_temp)
            parent_new_node.bounds = parent_new_node.cal_area()
            return node_temp

    def splitNode(self, d, n):
        def sort_by_bounds(item):
            return item.bounds[0]
        def sort_by_bounds_first_item(item):
            return item[0].bounds[0]
        if not self.is_leaf:
            node_new = RTreeNode(is_leaf=False)
            info = self.children + [(None, None)] * (d - len(self.children))
            self.children = sorted(info, key = sort_by_bounds)[:d // 2]
            node_new.children = sorted(info, key = sort_by_bounds)[d // 2:]
            for ch in self.children:
                ch.parent_node = self
            for ch in node_new.children:
                ch.parent_node = node_new
            node_new.cal_area()
            self.cal_area()
            return node_new
        else:
            node_new = RTreeNode(is_leaf=True)
            info= self.polygons + [(None, None)] * (n - len(self.polygons))
            self.polygons = sorted(info, key = sort_by_bounds_first_item)[:n // 2]
            node_new.polygons = sorted(info, key = sort_by_bounds_first_item)[n // 2:]
            node_new.cal_area()
            self.cal_area()
            return node_new

    def choose_leaf(self, polygons, index, d, n):
        child = self.children[0]
        min_increase = child.mbr_increase(polygons)
        for c in self.children[1:]:
            increase = c.mbr_increase(polygons)
            if increase < min_increase:
                child = c
                min_increase = increase
        child.insert(polygons, index, d, n)

    def cal_area(self):
        if self.is_leaf:
            bounds_lst = [bbox[0].bounds for bbox in self.polygons]
        else:
            bounds_lst = [bbox.bounds for bbox in self.children]

        xmin = bounds_lst[0][0]
        ymin = bounds_lst[0][1]
        xmax = bounds_lst[0][2]
        ymax = bounds_lst[0][3]
        for bounds in bounds_lst[1:]:
            xmin = min(xmin, bounds[0])
            ymin = min(ymin, bounds[1])
            xmax = max(xmax, bounds[2])
            ymax = max(ymax, bounds[3])
        return xmin, ymin, xmax, ymax

    def mbr_increase(self, polygons):
        bounds = [self.bounds, polygons.bounds]
        mbr_old=(self.bounds[2] - self.bounds[0]) * (self.bounds[3] - self.bounds[1])
        xmin = bounds[0][0]
        ymin = bounds[0][1]
        xmax = bounds[0][2]
        ymax = bounds[0][3]
        for b in bounds[1:]:
            xmin = min(xmin, b[0])
            ymin = min(ymin, b[1])
            xmax = max(xmax, b[2])
            ymax = max(ymax, b[3])
        return (xmax - xmin) * (ymax - ymin) - mbr_old

    def cal_height(self):
        height = -33
        if self.is_leaf:
            return 1
        else:
            for child in self.children:
                height = max(child.cal_height(), height)
            return height + 1

    def cal_leaf(self):
        if self.is_leaf:
            return 1
        else:
            total = 0
            for child in self.children:
                total += child.cal_leaf()
            return total

    def cal_non_leaf(self):
        if self.is_leaf:
            return 0
        else:
            total = 0
            for child in self.children:
                total += child.cal_non_leaf()
            return 1 + total


In [7]:
class RTreeIndex:
    def __init__(self, d, n):
        self.n = n
        self.d = d
        self.root = RTreeNode(is_leaf=True)
    def query_repeat(self, node, query, result_t):
        if not node.is_leaf:
            for child in node.children:
                if child.intersection(query):
                    self.query_repeat(child, query, result_t)
        else:
            for poly, idx in node.polygons:
                min_x, min_y, max_x, max_y = query.bounds
                x_min_p, y_min_p, x_max_p, y_max_p = poly.bounds
                if x_min_p >= min_x and y_min_p >= min_y and x_max_p <= max_x and y_max_p <= max_y:
                    result_t.append(idx)

    def query(self, query):
        result = []
        self.query_repeat(self.root, query, result)
        return result
    def cal_leaf(self):
        return self.root.cal_leaf()
    def cal_non_leaf(self):
        return self.root.cal_non_leaf()
    def cal_height(self):
        return self.root.cal_height()
    def insert(self, polygons, idx):
        self.root.insert(polygons, idx, self.d, self.n)
        while self.root.parent_node is not None:
            self.root = self.root.parent_node

Task 1

In [8]:
#Assignment 2 Task1
#Insert first-half of dataset D into R-Tree with n = 64, d = 8
tree_half_64 = RTreeIndex(8, 64)
for i,polygon in enumerate(polygon_list[:len(polygon_list)//2-1]):
    tree_half_64.insert(polygon, i)
print("Half of Dataset D with n = 64 and d = 8")
print("Height of R-tree: {}".format(tree_half_64.cal_height()))
print("Number of leaf Nodes: {}".format(tree_half_64.cal_leaf()))
print("Number of non_leaf nodes: {}".format(tree_half_64.cal_non_leaf()))

Half of Dataset D with n = 64 and d = 8
Height of R-tree: 5
Number of leaf Nodes: 1014
Number of non_leaf nodes: 242


In [9]:
#Insert all dataset D into R-Tree with n = 64, d = 8
tree_64 = RTreeIndex(8, 64)
for i,polygon in enumerate(polygon_list):
    tree_64.insert(polygon, i)
print("Dataset D with n = 64 and d = 8")
print("Height of R-tree: {}".format(tree_64.cal_height()))
print("Number of leaf Nodes: {}".format(tree_64.cal_leaf()))
print("Number of non_leaf nodes: {}".format(tree_64.cal_non_leaf()))

KeyboardInterrupt: 

In [ ]:
#Insert first-half of dataset D into R-Tree with n = 256, d = 32
tree_half_256 = RTreeIndex(32, 256)
for i,polygon in enumerate(polygon_list[:len(polygon_list)//2-1]):
    tree_half_256.insert(polygon, i)
print("Half of Dataset D with n = 256 and d = 32")
print("Height of R-tree: {}".format(tree_half_256.cal_height()))
print("Number of leaf Nodes: {}".format(tree_half_256.cal_leaf()))
print("Number of non_leaf nodes: {}".format(tree_half_256.cal_non_leaf()))

In [ ]:
#Insert all dataset D into R-Tree with n = 256, d = 32
tree_256 = RTreeIndex(32, 256)
for i,polygon in enumerate(polygon_list):
    tree_256.insert(polygon, i)
print("Dataset D with n = 256 and d = 32")
print("Height of R-tree: {}".format(tree_256.cal_height()))
print("Number of leaf Nodes: {}".format(tree_256.cal_leaf()))
print("Number of non_leaf nodes: {}".format(tree_256.cal_non_leaf()))

In [ ]:
tree_8_256 = RTreeIndex(8, 256)
for i,polygon in enumerate(polygon_list):
    tree_8_256.insert(polygon, i)
time_ex_list = []
time_rt_list = []
for j in range(0, 30):
    x_min = random.uniform(MBR_for_all.x_min,MBR_for_all.x_max)
    x_max = random.uniform(x_min,MBR_for_all.x_max)
    y_min = random.uniform(MBR_for_all.y_min,MBR_for_all.y_max)
    y_max = random.uniform(y_min,MBR_for_all.y_max)
    mbr_Q = MBR(x_min, y_min, x_max, y_max)
    query_window = Polygon([(x_min, y_min), (x_min, y_max), (x_max, y_min), (x_max, y_max)])
    count = 0
    # Exhaustive search method
    start_time_ex = time.time()
    for mbr_list in MBR_result:
        if is_contain(mbr_list, mbr_Q):
            count += 1
    end_time_ex = time.time()
    time_ex = end_time_ex - start_time_ex
    time_ex_list.append(time_ex)
    # R-Tree search method
    start_time_rt = time.time()
    result = tree_8_256.query(query_window)
    end_time_rt = time.time()
    time_rt = end_time_rt - start_time_rt
    time_rt_list.append(time_rt)
    print("Query {} Run_time for Exhaustive search method: {}".format(j + 1,time_ex))
    print("Query {} Exhaustive search: There are {} objects in Object_Q {}".format(j + 1, count, j + 1))
    print("Query {} Run_time for R-Tree search method: {}".format(j + 1,time_rt))
    print("Query {} R-Tree search: There are {} objects in Object_Q ".format(j + 1,len(result),j+1))

In [ ]:
min_time_ex = min(time_ex_list)
max_time_ex = max(time_ex_list)

avg_time_ex = sum(time_ex_list) / len(time_ex_list)
print("Minimum time to run Exhaustive search method: {}".format(min_time_ex))
print("Maximum time to run Exhaustive search method: {}".format(max_time_ex))
print("Average time to run Exhaustive search method: {}".format(avg_time_ex))

In [ ]:
min_time_rtree = min(time_rt_list)
max_time_rtree = max(time_rt_list)

avg_time_rtree = sum(time_rt_list) / len(time_rt_list)
print("Minimum time to run rtree search method: {}".format(min_time_rtree))
print("Maximum time to run rtree search method: {}".format(max_time_rtree))
print("Average time to run rtree search method: {}".format(avg_time_rtree))